In [1]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import platform
import shutil
import os.path
import fileinput

options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Windows'):
    options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),**options) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)

def replace(file,pattern,replace):
    fileinput.close()
    for line in fileinput.input(file, inplace=True):
        print( re.sub(pattern,
                      replace,
                      line.rstrip()
                      ) 
             )
if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())    

# Automated calibration for the GOTM/FABM model at Langtjern

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook

## Creating an instance on Google Cloud

In [2]:
instanceName = 'gotm-cal'
username = 'jose-luis'

createInstance = '''\
gcloud compute instances create {} \
--zone europe-west3-a \
--image-family debian-9 \
--image-project debian-cloud \
--machine-type n1-standard-2 \
'''
#--boot-disk-size 200GB \

deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''

if (platform.system() == 'Linux'):
    keyDir = ('/home/jose-luis/.ssh/gotmKeys')

if (platform.system() == 'Windows'):
    keyDir = ('c:/Users/jose_luis_guerrero/gotmKeys')
    

ip=''
instanceExists,ip = isInstance(instanceName)

if instanceExists:
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    if os.path.exists(keyDir):
        shutil.rmtree(keyDir)
    os.mkdir(keyDir)
    generateSSHKey(username,keyDir)
    keyFile = os.path.join(keyDir,username + '.pub')
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    #callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    replace(keyFile,r"^{}:".format(username),"")
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

gcloud compute instances create gotm-cal --zone europe-west3-a --image-family debian-9 --image-project debian-cloud --machine-type n1-standard-2 
NAME      ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
gotm-cal  europe-west3-a  n1-standard-2               10.156.0.4   35.234.95.92  RUNNING
Created [https://www.googleapis.com/compute/v1/projects/nivacatchment/zones/europe-west3-a/instances/gotm-cal].
Successfully created key pair
gcloud compute instances add-metadata gotm-cal --zone europe-west3-a --metadata-from-file ssh-keys=c:/Users/jose_luis_guerrero/gotmKeys/jose-luis.pub
Updated [https://www.googleapis.com/compute/v1/projects/nivacatchment/zones/europe-west3-a/instances/gotm-cal].
The ip of gotm-cal is 35.234.95.92


## Updating fabfile.py with credentials and ip

In [3]:
if (platform.system() == "Linux"):
    callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
    callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
    callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
    callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

fabfile = os.path.join("C:\\Users\\jose_luis_guerrero\\Envs\\mylai\\prognos_calibration","fabfile.py")
    
if (platform.system() == "Windows"):    
    replace(fabfile, "^env\.hosts.*",         "env.hosts=['{}']".format(ip))
    replace(fabfile, "^env\.user.*",          "env.user='{}'".format(username))
    replace(fabfile, "^env\.key_filename.*",  "env.key_filename='{}'".format(os.path.join(keyDir,username)))
    replace(fabfile, "^env\.roledefs.*",      "env.roledefs={{'{}':['{}'],".format('stage',ip))

#Testing connection
#Adding key to remote machine
p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
print(p.communicate())

(b'', None)


## Setting up GOTM and FABM in the remote machine

In [6]:
#callPopen('fab testConnection')
callPopen('fab getUtilities')
callPopen('fab downloadModels') #note that the location of the key file is hardcoded in the getFABM function
#callPopen('fab compileModels')
#callPopen('fab getPROGNOS')
#callPopen('fab testRun:/home/{}/PROGNOS/langtjern/langtjern.xml'.format(username))

fab getUtilities
[35.234.95.92] Executing task 'getUtilities'
[35.234.95.92] Executing task 'update'
[35.234.95.92] Executing task 'updateMachine'
[35.234.95.92] run: sudo apt-get update
[35.234.95.92] out: 
[35.234.95.92] out: 0% [Working]
[35.234.95.92] out:             
[35.234.95.92] out: Ign:1 http://deb.debian.org/debian stretch InRelease
[35.234.95.92] out: 
[35.234.95.92] out: 0% [Waiting for headers] [Connecting to packages.cloud.google.com (172.217.21.2
[35.234.95.92] out:                                                                                
[35.234.95.92] out: Hit:2 http://security.debian.org stretch/updates InRelease
[35.234.95.92] out: 
[35.234.95.92] out: 0% [Waiting for headers] [Connecting to packages.cloud.google.com (172.217.21.2
[35.234.95.92] out:                                                                                
[35.234.95.92] out: Hit:3 http://deb.debian.org/debian stretch-updates InRelease
[35.234.95.92] out: 
[35.234.95.92] out:           


[35.234.95.92] Executing task 'getModules'
[35.234.95.92] run: sudo pip install xmlstore editscenario xmlplot matplotlib
[35.234.95.92] out: Collecting xmlstore
[35.234.95.92] out:   Using cached https://files.pythonhosted.org/packages/51/77/5dcefa8336915fd721f06f6581552bd4a3d43fbd04c83edd7218edbd3f74/xmlstore-0.9.4-py2-none-any.whl
[35.234.95.92] out: Collecting editscenario
[35.234.95.92] out:   Using cached https://files.pythonhosted.org/packages/80/64/06d359c6c649de2f603305196b1a8f9c3b4610968f373314c35756547016/editscenario-0.11.1-py2-none-any.whl
[35.234.95.92] out: Collecting xmlplot
[35.234.95.92] out:   Downloading https://files.pythonhosted.org/packages/cd/e0/e9f674e73bcc89db87f5edcab241e06eb24d28d4c24b526043d06b646408/xmlplot-0.9.9-py2-none-any.whl (156kB)
[35.234.95.92] out: 
[35.234.95.92] out: Disconnecting from 35.234.95.92... done.
Traceback (most recent call last):
  File "c:\users\jose_luis_guerrero\envs\mylai\lib\site-packages\fabric\main.py", line 763, in main
    *